# atmodeller

## Tutorial 4: Interior (include Si in the atmosphere)

Import the required functionality and activate the debug logger.

In [23]:
from atmodeller import debug_logger, debug_file_logger
from atmodeller.constraints import  MassConstraint, SystemConstraints, IronWustiteBufferConstraintHirschmann, FugacityConstraint
from atmodeller.interior_atmosphere import InteriorAtmosphereSystem, Planet, Species
from atmodeller.interfaces import GasSpecies, LiquidSpecies
from atmodeller.solubilities import PeridotiteH2O, BasaltDixonCO2
from atmodeller.utilities import earth_oceans_to_kg

# debug_file_logger()
debug_logger()

### 1. H2O-H2-O2-SiO-SiH4-SiO2l system

We define a list of the species we wish to include in the interior-atmosphere system

In [24]:
species: Species = Species()
species.append(GasSpecies(chemical_formula='H2O'))#, solubility=PeridotiteH2O()))
species.append(GasSpecies(chemical_formula='H2'))
species.append(GasSpecies(chemical_formula='O2'))
species.append(GasSpecies(chemical_formula='OSi'))
species.append(GasSpecies(chemical_formula='H4Si'))
species.append(LiquidSpecies(chemical_formula="O2Si",
                             name_in_thermodynamic_data="O2Si(l)",
                             #thermodynamic_class=thermodynamic_data,
                            ))


[15:38:17 - atmodeller.interfaces          - INFO     ] - Creating a GasSpecies: H2O (H2O)
[15:38:17 - atmodeller.interfaces          - DEBUG    ] - Modified Hill formula = H2O
[15:38:17 - atmodeller.interfaces          - DEBUG    ] - Thermodynamic data for H2O (H2O) = <thermochem.janaf.JanafPhase object at 0x7fe058798f10>
  Water (H2O)	H2O1(g)
    Cp(298.15) = 33.590 J/mol/K
    S(298.15) = 188.834 J/mol/K
    [G-H(298.15)]/298.15 = 188.834 J/mol/K
    H-H(298.15) = 0.000 J/mol/K
    Delta_fH(298.15) = -241826 J/mol
    Delta_fG(298.15) = -228582 J/mol
    log(Kf((298.15)) = 40.047
[15:38:17 - atmodeller.interfaces          - INFO     ] - Creating a GasSpecies: H2 (H2)
[15:38:17 - atmodeller.interfaces          - DEBUG    ] - Modified Hill formula = H2
[15:38:17 - atmodeller.interfaces          - DEBUG    ] - Thermodynamic data for H2 (H2) = <thermochem.janaf.JanafPhase object at 0x7fe05879a590>
  Hydrogen (H2)	H2(ref)
    Cp(298.15) = 28.836 J/mol/K
    S(298.15) = 130.680 J/mol/K
  

Create an interior-atmosphere system using H-O-Si for a sub-Neptune (Planet mass = 11 M_Earth, Interior radius = 1.7 R_Earth)

In [25]:
planet = Planet(surface_temperature=3000, mantle_mass=11*4.208261222595111e24, surface_radius=1.7*6371000)
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

[15:38:19 - atmodeller.interior_atmosphere - INFO     ] - Creating a new planet
[15:38:19 - atmodeller.interior_atmosphere - INFO     ] - Mantle mass (kg) = 46290873448546219736432640.000000
[15:38:19 - atmodeller.interior_atmosphere - INFO     ] - Mantle melt fraction = 1.000000
[15:38:19 - atmodeller.interior_atmosphere - INFO     ] - Core mass fraction = 0.295335
[15:38:19 - atmodeller.interior_atmosphere - INFO     ] - Planetary radius (m) = 10830700.000000
[15:38:19 - atmodeller.interior_atmosphere - INFO     ] - Planetary mass (kg) = 65692000000000001451229184.000000
[15:38:19 - atmodeller.interior_atmosphere - INFO     ] - Surface temperature (K) = 3000.000000
[15:38:19 - atmodeller.interior_atmosphere - INFO     ] - Surface gravity (m/s^2) = 37.377061
[15:38:19 - atmodeller.interior_atmosphere - INFO     ] - Melt Composition = None
[15:38:19 - atmodeller.interior_atmosphere - INFO     ] - Creating an interior-atmosphere system
[15:38:19 - atmodeller.interior_atmosphere - INFO  

To solve the system, we provide a constraint of the H2O and O2 fugacity in bar and assume the actvity of SiO2(l)=1.

In [47]:
constraints: SystemConstraints = SystemConstraints(
    [
        FugacityConstraint(species='H2', value=10000), # This constraint sets surface pressure of approx 10 kbar (1 GPa)
        FugacityConstraint(species='O2Si', value=1),  # This constraints simulates activity of 1
        IronWustiteBufferConstraintHirschmann(log10_shift=-4), # This constraint sets a low fO2 simulating a sub-Neptune surface
    ])


In [27]:
interior_atmosphere.solve(constraints)

[15:38:23 - atmodeller.interior_atmosphere - INFO     ] - Assembling constraints
[15:38:23 - atmodeller.interior_atmosphere - INFO     ] - Constraints: [FugacityConstraint(name='fugacity', species='H2', value=10000),
 FugacityConstraint(name='fugacity', species='O2Si', value=1),
 IronWustiteBufferConstraintHirschmann(name='fugacity',
                                       species='O2',
                                       log10_shift=-4)]
[15:38:23 - atmodeller.interior_atmosphere - DEBUG    ] - Conforming initial solution to solid activities = [1. 1. 1. 1. 1. 1.]
[15:38:23 - atmodeller.interior_atmosphere - DEBUG    ] - Setting H2 1
[15:38:23 - atmodeller.interior_atmosphere - DEBUG    ] - Setting O2Si 5
[15:38:23 - atmodeller.interior_atmosphere - DEBUG    ] - Setting O2 2
[15:38:23 - atmodeller.interior_atmosphere - DEBUG    ] - Conforming initial solution to constraints = [ 1.          4.         -6.62323054  1.          1.          0.        ]
[15:38:23 - atmodeller.interior_atm

You can access the solution directly using:

In [28]:
interior_atmosphere.solution_dict

{'H2O': 133.87899873047937,
 'H2': 10000.0,
 'O2': 3.684877247534094e-07,
 'OSi': 136.32016786085802,
 'H4Si': 79.54079537171324,
 'O2Si': 1.0}

You can access more information about the solution using:

In [13]:
interior_atmosphere.output

{'temperature': 3000,
 'total_pressure_in_atmosphere': 11232.566902658169,
 'mean_molar_mass_in_atmosphere': 0.005715615826546348,
 'H2O': GasSpeciesOutput(mass_in_atmosphere=5.361680771744624e+20, mass_in_solid=0.0, mass_in_melt=0.0, moles_in_atmosphere=2.976183932981264e+22, moles_in_melt=0.0, moles_in_solid=0.0, ppmw_in_solid=0.0, ppmw_in_melt=0.0, fugacity=43.13254021750895, fugacity_coefficient=1.0, pressure_in_atmosphere=43.13254021750895, volume_mixing_ratio=0.003839954000834992, mass_in_total=5.361680771744624e+20),
 'H2': GasSpeciesOutput(mass_in_atmosphere=1.3909766475452525e+22, mass_in_solid=0.0, mass_in_melt=0.0, moles_in_atmosphere=6.900089626006147e+24, moles_in_melt=0.0, moles_in_solid=0.0, ppmw_in_solid=0.0, ppmw_in_melt=0.0, fugacity=10000.0, fugacity_coefficient=1.0, pressure_in_atmosphere=10000.0, volume_mixing_ratio=0.8902684565923676, mass_in_total=1.3909766475452525e+22),
 'O2': GasSpeciesOutput(mass_in_atmosphere=844494343824.7837, mass_in_solid=0.0, mass_in_mel

### 2. H2O-H2-O2-SiO-SiH4-SiO2l-CH4-CO-CO2 system

We now extend the species list to additionally include C-species:

In [30]:
species: Species = Species()
species.append(GasSpecies(chemical_formula='H2O'))
species.append(GasSpecies(chemical_formula='H2'))
species.append(GasSpecies(chemical_formula='O2'))
species.append(GasSpecies(chemical_formula='CH4'))
species.append(GasSpecies(chemical_formula='CO'))
species.append(GasSpecies(chemical_formula='CO2'))
species.append(GasSpecies(chemical_formula='OSi'))
species.append(GasSpecies(chemical_formula='H4Si'))
species.append(LiquidSpecies(chemical_formula="O2Si",
                             name_in_thermodynamic_data="O2Si(l)",
                             #thermodynamic_class=thermodynamic_data,
                            ))

species

[15:39:52 - atmodeller.interfaces          - INFO     ] - Creating a GasSpecies: H2O (H2O)
[15:39:52 - atmodeller.interfaces          - DEBUG    ] - Modified Hill formula = H2O
[15:39:52 - atmodeller.interfaces          - DEBUG    ] - Thermodynamic data for H2O (H2O) = <thermochem.janaf.JanafPhase object at 0x7fe0587ad690>
  Water (H2O)	H2O1(g)
    Cp(298.15) = 33.590 J/mol/K
    S(298.15) = 188.834 J/mol/K
    [G-H(298.15)]/298.15 = 188.834 J/mol/K
    H-H(298.15) = 0.000 J/mol/K
    Delta_fH(298.15) = -241826 J/mol
    Delta_fG(298.15) = -228582 J/mol
    log(Kf((298.15)) = 40.047
[15:39:52 - atmodeller.interfaces          - INFO     ] - Creating a GasSpecies: H2 (H2)
[15:39:52 - atmodeller.interfaces          - DEBUG    ] - Modified Hill formula = H2
[15:39:52 - atmodeller.interfaces          - DEBUG    ] - Thermodynamic data for H2 (H2) = <thermochem.janaf.JanafPhase object at 0x7fe058799bd0>
  Hydrogen (H2)	H2(ref)
    Cp(298.15) = 28.836 J/mol/K
    S(298.15) = 130.680 J/mol/K
  

Species([GasSpecies(chemical_formula='H2O', name_in_thermodynamic_data='H2O', thermodynamic_class=<class 'atmodeller.interfaces.ThermodynamicDataJANAF'>, formula=Formula('H2O'), thermodynamic_data=<atmodeller.interfaces.ThermodynamicDataJANAF object at 0x7fe0587ad630>, output=None, solubility=<atmodeller.interfaces.NoSolubility object at 0x7fe0588b43a0>, solid_melt_distribution_coefficient=0, fugacity_coefficient=IdealityConstant(name='', species='', value=1.0)),
         GasSpecies(chemical_formula='H2', name_in_thermodynamic_data='H2', thermodynamic_class=<class 'atmodeller.interfaces.ThermodynamicDataJANAF'>, formula=Formula('H2'), thermodynamic_data=<atmodeller.interfaces.ThermodynamicDataJANAF object at 0x7fe0aa651090>, output=None, solubility=<atmodeller.interfaces.NoSolubility object at 0x7fe0aa651c30>, solid_melt_distribution_coefficient=0, fugacity_coefficient=IdealityConstant(name='', species='', value=1.0)),
         GasSpecies(chemical_formula='O2', name_in_thermodynamic_da

In [31]:
planet = Planet(surface_temperature=3000, mantle_mass=11*4.208261222595111e24, surface_radius=1.7*6371000)
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

[15:39:55 - atmodeller.interior_atmosphere - INFO     ] - Creating a new planet
[15:39:55 - atmodeller.interior_atmosphere - INFO     ] - Mantle mass (kg) = 46290873448546219736432640.000000
[15:39:55 - atmodeller.interior_atmosphere - INFO     ] - Mantle melt fraction = 1.000000
[15:39:55 - atmodeller.interior_atmosphere - INFO     ] - Core mass fraction = 0.295335
[15:39:55 - atmodeller.interior_atmosphere - INFO     ] - Planetary radius (m) = 10830700.000000
[15:39:55 - atmodeller.interior_atmosphere - INFO     ] - Planetary mass (kg) = 65692000000000001451229184.000000
[15:39:55 - atmodeller.interior_atmosphere - INFO     ] - Surface temperature (K) = 3000.000000
[15:39:55 - atmodeller.interior_atmosphere - INFO     ] - Surface gravity (m/s^2) = 37.377061
[15:39:55 - atmodeller.interior_atmosphere - INFO     ] - Melt Composition = None
[15:39:55 - atmodeller.interior_atmosphere - INFO     ] - Creating an interior-atmosphere system
[15:39:55 - atmodeller.interior_atmosphere - INFO  

An additional constraint is required to solve the system

In [32]:
constraints: SystemConstraints = SystemConstraints(
    [
        FugacityConstraint(species='H2', value=10000), # This constraint sets surface pressure of 10 kbar (1 GPa)
        FugacityConstraint(species='O2Si', value=1),  # This constraints simulates activity of 1
        FugacityConstraint(species='CO', value=1),  # This is an arbitrary constraint
        IronWustiteBufferConstraintHirschmann(log10_shift=-4),  # This constraint sets a low fO2 simulating a sub-Neptune surface
    ])


In [33]:
interior_atmosphere.solve(constraints)
interior_atmosphere.solution_dict

[15:39:59 - atmodeller.interior_atmosphere - INFO     ] - Assembling constraints
[15:39:59 - atmodeller.interior_atmosphere - INFO     ] - Constraints: [FugacityConstraint(name='fugacity', species='H2', value=10000),
 FugacityConstraint(name='fugacity', species='O2Si', value=1),
 FugacityConstraint(name='fugacity', species='CO', value=1),
 IronWustiteBufferConstraintHirschmann(name='fugacity',
                                       species='O2',
                                       log10_shift=-4)]
[15:39:59 - atmodeller.interior_atmosphere - DEBUG    ] - Conforming initial solution to solid activities = [1. 1. 1. 1. 1. 1. 1. 1. 1.]
[15:39:59 - atmodeller.interior_atmosphere - DEBUG    ] - Setting H2 1
[15:39:59 - atmodeller.interior_atmosphere - DEBUG    ] - Setting O2Si 8
[15:39:59 - atmodeller.interior_atmosphere - DEBUG    ] - Setting CO 4
[15:39:59 - atmodeller.interior_atmosphere - DEBUG    ] - Setting O2 2
[15:39:59 - atmodeller.interior_atmosphere - DEBUG    ] - Conforming in

{'H2O': 133.8928569794622,
 'H2': 10000.0,
 'O2': 3.6856401541864007e-07,
 'CH4': 3.9188304375948766,
 'CO': 1.0,
 'CO2': 0.0018382948171367357,
 'OSi': 136.3060583790664,
 'H4Si': 79.52433087724863,
 'O2Si': 1.0}

### 3. System with mixed constraints

A typical use case is to define an interior-atmosphere system with a combination of fugacity and mass constraints. We define the same species as before:

In [77]:
species: Species = Species()
species.append(GasSpecies(chemical_formula='H2O'))
species.append(GasSpecies(chemical_formula='H2'))
species.append(GasSpecies(chemical_formula='O2'))
species.append(GasSpecies(chemical_formula='CH4'))
species.append(GasSpecies(chemical_formula='CO'))
species.append(GasSpecies(chemical_formula='CO2'))
species.append(GasSpecies(chemical_formula='OSi'))
species.append(GasSpecies(chemical_formula='H4Si'))
species.append(LiquidSpecies(chemical_formula="O2Si",
                             name_in_thermodynamic_data="O2Si(l)",
                             #thermodynamic_class=thermodynamic_data,
                            ))

species

[16:09:59 - atmodeller.interfaces          - INFO     ] - Creating a GasSpecies: H2O (H2O)
[16:09:59 - atmodeller.interfaces          - DEBUG    ] - Modified Hill formula = H2O
[16:09:59 - atmodeller.interfaces          - DEBUG    ] - Thermodynamic data for H2O (H2O) = <thermochem.janaf.JanafPhase object at 0x7fe0589cab60>
  Water (H2O)	H2O1(g)
    Cp(298.15) = 33.590 J/mol/K
    S(298.15) = 188.834 J/mol/K
    [G-H(298.15)]/298.15 = 188.834 J/mol/K
    H-H(298.15) = 0.000 J/mol/K
    Delta_fH(298.15) = -241826 J/mol
    Delta_fG(298.15) = -228582 J/mol
    log(Kf((298.15)) = 40.047
[16:09:59 - atmodeller.interfaces          - INFO     ] - Creating a GasSpecies: H2 (H2)
[16:09:59 - atmodeller.interfaces          - DEBUG    ] - Modified Hill formula = H2
[16:09:59 - atmodeller.interfaces          - DEBUG    ] - Thermodynamic data for H2 (H2) = <thermochem.janaf.JanafPhase object at 0x7fe058a01e10>
  Hydrogen (H2)	H2(ref)
    Cp(298.15) = 28.836 J/mol/K
    S(298.15) = 130.680 J/mol/K
  

Species([GasSpecies(chemical_formula='H2O', name_in_thermodynamic_data='H2O', thermodynamic_class=<class 'atmodeller.interfaces.ThermodynamicDataJANAF'>, formula=Formula('H2O'), thermodynamic_data=<atmodeller.interfaces.ThermodynamicDataJANAF object at 0x7fe0aa6538b0>, output=None, solubility=<atmodeller.interfaces.NoSolubility object at 0x7fe058845e40>, solid_melt_distribution_coefficient=0, fugacity_coefficient=IdealityConstant(name='', species='', value=1.0)),
         GasSpecies(chemical_formula='H2', name_in_thermodynamic_data='H2', thermodynamic_class=<class 'atmodeller.interfaces.ThermodynamicDataJANAF'>, formula=Formula('H2'), thermodynamic_data=<atmodeller.interfaces.ThermodynamicDataJANAF object at 0x7fe0589c8df0>, output=None, solubility=<atmodeller.interfaces.NoSolubility object at 0x7fe0aa70b2b0>, solid_melt_distribution_coefficient=0, fugacity_coefficient=IdealityConstant(name='', species='', value=1.0)),
         GasSpecies(chemical_formula='O2', name_in_thermodynamic_da

Now we define the constraints, and in this case we want to constrain the total mass of C and H in the system that can partition between the various reservoirs.

In [78]:
number_of_earth_oceans: float = 11 # 11x to account for mass increase by 11x
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = earth_oceans_to_kg(number_of_earth_oceans) # note: this constraint is not used below
mass_C: float = ch_ratio * mass_H

constraints: SystemConstraints = SystemConstraints([
    FugacityConstraint(species='H2', value=10000), # This constraint sets surface pressure of 10 kbar (1 GPa)
    # MassConstraint(species="H", value=mass_H), # Mass constraint of H
    MassConstraint(species="C", value=mass_C), # Mass constraint of C
    FugacityConstraint(species='O2Si', value=1), # Simulates an activity of unity
    IronWustiteBufferConstraintHirschmann(log10_shift=-4) # Sets a low fO2 for a sub-Neptune surface
])

In [79]:
planet = Planet(surface_temperature=3000, mantle_mass=11*4.208261222595111e24, surface_radius=1.7*6371000)
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

[16:10:03 - atmodeller.interior_atmosphere - INFO     ] - Creating a new planet
[16:10:03 - atmodeller.interior_atmosphere - INFO     ] - Mantle mass (kg) = 46290873448546219736432640.000000
[16:10:03 - atmodeller.interior_atmosphere - INFO     ] - Mantle melt fraction = 1.000000
[16:10:03 - atmodeller.interior_atmosphere - INFO     ] - Core mass fraction = 0.295335
[16:10:03 - atmodeller.interior_atmosphere - INFO     ] - Planetary radius (m) = 10830700.000000
[16:10:03 - atmodeller.interior_atmosphere - INFO     ] - Planetary mass (kg) = 65692000000000001451229184.000000
[16:10:03 - atmodeller.interior_atmosphere - INFO     ] - Surface temperature (K) = 3000.000000
[16:10:03 - atmodeller.interior_atmosphere - INFO     ] - Surface gravity (m/s^2) = 37.377061
[16:10:03 - atmodeller.interior_atmosphere - INFO     ] - Melt Composition = None
[16:10:03 - atmodeller.interior_atmosphere - INFO     ] - Creating an interior-atmosphere system
[16:10:03 - atmodeller.interior_atmosphere - INFO  

Solution

In [80]:
interior_atmosphere.solve(constraints)
interior_atmosphere.solution_dict

[16:10:06 - atmodeller.interior_atmosphere - INFO     ] - Assembling constraints
[16:10:06 - atmodeller.interior_atmosphere - INFO     ] - Constraints: [FugacityConstraint(name='fugacity', species='H2', value=10000),
 MassConstraint(name='mass', species='C', value=1.7050016915689426e+21),
 FugacityConstraint(name='fugacity', species='O2Si', value=1),
 IronWustiteBufferConstraintHirschmann(name='fugacity',
                                       species='O2',
                                       log10_shift=-4)]
[16:10:06 - atmodeller.interior_atmosphere - DEBUG    ] - Conforming initial solution to solid activities = [1. 1. 1. 1. 1. 1. 1. 1. 1.]
[16:10:06 - atmodeller.interior_atmosphere - DEBUG    ] - Setting H2 1
[16:10:06 - atmodeller.interior_atmosphere - DEBUG    ] - Setting O2Si 8
[16:10:06 - atmodeller.interior_atmosphere - DEBUG    ] - Setting O2 2
[16:10:06 - atmodeller.interior_atmosphere - DEBUG    ] - Conforming initial solution to constraints = [ 1.          4.         -6

{'H2O': 134.2012847722831,
 'H2': 10000.0,
 'O2': 3.7026397636289936e-07,
 'CH4': 90.98707109769467,
 'CO': 23.271398207070742,
 'CO2': 0.042878235517343925,
 'OSi': 135.99279329516423,
 'H4Si': 79.15921770059694,
 'O2Si': 1.0}

### 4. System with mixed constraints and H2O-CO2 solubilities

Include H2O and CO2 solubilities

In [81]:
species: Species = Species()
species.append(GasSpecies(chemical_formula='H2O', solubility=PeridotiteH2O()))
species.append(GasSpecies(chemical_formula='H2'))
species.append(GasSpecies(chemical_formula='O2'))
species.append(GasSpecies(chemical_formula='CH4'))
species.append(GasSpecies(chemical_formula='CO'))
species.append(GasSpecies(chemical_formula='CO2', solubility=BasaltDixonCO2()))
species.append(GasSpecies(chemical_formula='OSi'))
species.append(GasSpecies(chemical_formula='H4Si'))
species.append(LiquidSpecies(chemical_formula="O2Si",
                             name_in_thermodynamic_data="O2Si(l)",
                             #thermodynamic_class=thermodynamic_data,
                            ))

species

[16:10:16 - atmodeller.interfaces          - INFO     ] - Creating a GasSpecies: H2O (H2O)
[16:10:16 - atmodeller.interfaces          - DEBUG    ] - Modified Hill formula = H2O
[16:10:16 - atmodeller.interfaces          - DEBUG    ] - Thermodynamic data for H2O (H2O) = <thermochem.janaf.JanafPhase object at 0x7fe058a133d0>
  Water (H2O)	H2O1(g)
    Cp(298.15) = 33.590 J/mol/K
    S(298.15) = 188.834 J/mol/K
    [G-H(298.15)]/298.15 = 188.834 J/mol/K
    H-H(298.15) = 0.000 J/mol/K
    Delta_fH(298.15) = -241826 J/mol
    Delta_fG(298.15) = -228582 J/mol
    log(Kf((298.15)) = 40.047
[16:10:16 - atmodeller.interfaces          - INFO     ] - Creating a GasSpecies: H2 (H2)
[16:10:16 - atmodeller.interfaces          - DEBUG    ] - Modified Hill formula = H2
[16:10:17 - atmodeller.interfaces          - DEBUG    ] - Thermodynamic data for H2 (H2) = <thermochem.janaf.JanafPhase object at 0x7fe058a00e20>
  Hydrogen (H2)	H2(ref)
    Cp(298.15) = 28.836 J/mol/K
    S(298.15) = 130.680 J/mol/K
  

Species([GasSpecies(chemical_formula='H2O', name_in_thermodynamic_data='H2O', thermodynamic_class=<class 'atmodeller.interfaces.ThermodynamicDataJANAF'>, formula=Formula('H2O'), thermodynamic_data=<atmodeller.interfaces.ThermodynamicDataJANAF object at 0x7fe058a11870>, output=None, solubility=<atmodeller.solubilities.PeridotiteH2O object at 0x7fe0589bf0d0>, solid_melt_distribution_coefficient=0, fugacity_coefficient=IdealityConstant(name='', species='', value=1.0)),
         GasSpecies(chemical_formula='H2', name_in_thermodynamic_data='H2', thermodynamic_class=<class 'atmodeller.interfaces.ThermodynamicDataJANAF'>, formula=Formula('H2'), thermodynamic_data=<atmodeller.interfaces.ThermodynamicDataJANAF object at 0x7fe05852ef50>, output=None, solubility=<atmodeller.interfaces.NoSolubility object at 0x7fe0589bc3d0>, solid_melt_distribution_coefficient=0, fugacity_coefficient=IdealityConstant(name='', species='', value=1.0)),
         GasSpecies(chemical_formula='O2', name_in_thermodynamic

In [82]:
number_of_earth_oceans: float = 11 # 11x to account for mass increase by 11x
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = earth_oceans_to_kg(number_of_earth_oceans) # note: this constraint is not used below
mass_C: float = ch_ratio * mass_H

constraints: SystemConstraints = SystemConstraints([
    FugacityConstraint(species='H2', value=10000), # This constraint sets surface pressure of 10 kbar (1 GPa)
    # MassConstraint(species="H", value=mass_H), # Mass constraint of H
    MassConstraint(species="C", value=mass_C), # Mass constraint of C
    FugacityConstraint(species='O2Si', value=1), # Simulates an activity of unity
    IronWustiteBufferConstraintHirschmann(log10_shift=-4) # Sets a low fO2 for a sub-Neptune surface
])

In [83]:
planet = Planet(surface_temperature=3000, mantle_mass=11*4.208261222595111e24, surface_radius=1.7*6371000)
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

[16:10:21 - atmodeller.interior_atmosphere - INFO     ] - Creating a new planet
[16:10:21 - atmodeller.interior_atmosphere - INFO     ] - Mantle mass (kg) = 46290873448546219736432640.000000
[16:10:21 - atmodeller.interior_atmosphere - INFO     ] - Mantle melt fraction = 1.000000
[16:10:21 - atmodeller.interior_atmosphere - INFO     ] - Core mass fraction = 0.295335
[16:10:21 - atmodeller.interior_atmosphere - INFO     ] - Planetary radius (m) = 10830700.000000
[16:10:21 - atmodeller.interior_atmosphere - INFO     ] - Planetary mass (kg) = 65692000000000001451229184.000000
[16:10:21 - atmodeller.interior_atmosphere - INFO     ] - Surface temperature (K) = 3000.000000
[16:10:21 - atmodeller.interior_atmosphere - INFO     ] - Surface gravity (m/s^2) = 37.377061
[16:10:21 - atmodeller.interior_atmosphere - INFO     ] - Melt Composition = None
[16:10:21 - atmodeller.interior_atmosphere - INFO     ] - Creating an interior-atmosphere system
[16:10:21 - atmodeller.interior_atmosphere - INFO  

In [84]:
interior_atmosphere.solve(constraints)
interior_atmosphere.solution_dict

[16:10:23 - atmodeller.interior_atmosphere - INFO     ] - Assembling constraints
[16:10:23 - atmodeller.interior_atmosphere - INFO     ] - Constraints: [FugacityConstraint(name='fugacity', species='H2', value=10000),
 MassConstraint(name='mass', species='C', value=1.7050016915689426e+21),
 FugacityConstraint(name='fugacity', species='O2Si', value=1),
 IronWustiteBufferConstraintHirschmann(name='fugacity',
                                       species='O2',
                                       log10_shift=-4)]
[16:10:23 - atmodeller.interior_atmosphere - DEBUG    ] - Conforming initial solution to solid activities = [1. 1. 1. 1. 1. 1. 1. 1. 1.]
[16:10:23 - atmodeller.interior_atmosphere - DEBUG    ] - Setting H2 1
[16:10:23 - atmodeller.interior_atmosphere - DEBUG    ] - Setting O2Si 8
[16:10:23 - atmodeller.interior_atmosphere - DEBUG    ] - Setting O2 2
[16:10:23 - atmodeller.interior_atmosphere - DEBUG    ] - Conforming initial solution to constraints = [ 1.          4.         -6

{'H2O': 134.2012353681171,
 'H2': 10000.0,
 'O2': 3.702637037488239e-07,
 'CH4': 90.97314722062642,
 'CO': 23.267828387183595,
 'CO2': 0.042871642236677006,
 'OSi': 135.99284335886534,
 'H4Si': 79.15927598315747,
 'O2Si': 1.0}